In [1]:
using BenchmarkTools

In [2]:
function MaxPool_v1(input, pool_size, gradient)
    input_height, input_width, channels = size(input)
    pool_height, pool_width = pool_size
    gradient_height, gradient_width = size(gradient)
    
    input_height % pool_height != 0 ? input_height_new = pool_height*size(gradient)[1] : input_height_new = input_height
    input_width % pool_width != 0 ? input_width_new = pool_width*size(gradient)[2] : input_width_new = input_width
    
    J = zeros(Float32, input_height, input_width, channels)
        
    for c in 1:channels
        for i in 1:pool_width:input_width_new
            for j in 1:pool_height:input_height_new
                end_i = min(i + pool_width - 1, input_width)
                end_j = min(j + pool_height - 1, input_height)
            
                max_value, max_idx = findmax(input[i:end_i, j:end_j,c])
                    
                J[i + max_idx[1] - 1, j + max_idx[2] - 1,c] = 1*gradient[div(i-1,pool_width) + 1, div(j-1, pool_height) + 1, c]
            end
        end
    end

    return tuple(J)
end


MaxPool_v1 (generic function with 1 method)

In [3]:
function MaxPool_v2(input, pool_size, gradient)
    input_height, input_width, channels = size(input)
    pool_height, pool_width = pool_size
    gradient_height, gradient_width = size(gradient)
    
    input_height % pool_height != 0 ? input_height_new = pool_height*size(gradient)[1] : input_height_new = input_height
    input_width % pool_width != 0 ? input_width_new = pool_width*size(gradient)[2] : input_width_new = input_width
    
    J = zeros(Float32, input_height, input_width, channels)
        
    for c in 1:channels
        for j in 1:pool_height:input_height_new
            for i in 1:pool_width:input_width_new
                end_i = min(i + pool_width - 1, input_width)
                end_j = min(j + pool_height - 1, input_height)
            
                max_value, max_idx = findmax(@views input[i:end_i, j:end_j,c])
                    
                J[i + max_idx[1] - 1, j + max_idx[2] - 1,c] = 1*gradient[div(i-1,pool_width) + 1, div(j-1, pool_height) + 1, c]
            end
        end
    end

    return tuple(J)
end

MaxPool_v2 (generic function with 1 method)

In [28]:
function MaxPool_v3(input, pool_size, gradient)
    input_height, input_width, channels = size(input)
    pool_height, pool_width = pool_size
    gradient_height, gradient_width = size(gradient)

    J = zeros(Float32, input_height, input_width, channels)
    
    input_height % pool_height != 0 ? input_height_new = pool_height*gradient_height : input_height_new = input_height
    input_width % pool_width != 0 ? input_width_new = pool_width*gradient_width : input_width_new = input_width
    
    
        
    for c in 1:channels
        for j in 1:pool_height:input_height_new
            for i in 1:pool_width:input_width_new
                end_i = min(i + pool_width - 1, input_width)
                end_j = min(j + pool_height - 1, input_height)
            
                @views max_value, max_idx = findmax(input[i:end_i, j:end_j,c])
                    
                J[i + max_idx[1] - 1, j + max_idx[2] - 1,c] = 1*gradient[div(i-1,pool_width) + 1, div(j-1, pool_height) + 1, c]
            end
        end
    end

    return tuple(J)
end

MaxPool_v3 (generic function with 1 method)

In [30]:
input = rand(Float32, 11, 11, 16);
gradient= rand(Float32, 5, 5, 16);

In [31]:
@benchmark MaxPool_v1(input, (2,2), gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  18.041 μs …  2.523 ms  ┊ GC (min … max): 0.00% … 98.20%
 Time  (median):     19.667 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.711 μs ± 55.216 μs  ┊ GC (mean ± σ):  7.05% ±  2.77%

   ▂▅▆████▇▇▆▅▅▄▃▂▁▁ ▁                                        ▃
  ████████████████████████▇▇▇▇▆██▇▇▆▆▇▆▇▆▆▆▆▆▆▆▆▆▅▄▅▅▅▅▅▆▄▅▅▆ █
  18 μs        Histogram: log(frequency) by time      31.3 μs <

 Memory estimate: 39.02 KiB, allocs estimate: 402.

In [32]:
@benchmark MaxPool_v2(input, (2,2), gradient)

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.489 μs … 960.844 μs  ┊ GC (min … max): 0.00% … 98.48%
 Time  (median):     8.396 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.878 μs ±  16.893 μs  ┊ GC (mean ± σ):  4.88% ±  2.59%

               ▄▃▇▆█▅▇▄▇▂▃▁▁                                   
  ▄█▅▅▄▄▃▃▃▃▄▆▆█████████████▇█▆▇▅▅▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  7.49 μs         Histogram: frequency by time        10.3 μs <

 Memory estimate: 7.77 KiB, allocs estimate: 2.

In [33]:
@benchmark MaxPool_v3(input, (2,2), gradient)

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.417 μs … 958.719 μs  ┊ GC (min … max): 0.00% … 98.39%
 Time  (median):     8.427 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.878 μs ±  16.266 μs  ┊ GC (mean ± σ):  4.65% ±  2.59%

             ▂▂▃▅▇█▇▆▅▃▁                                       
  ▄▆▆▅▃▃▂▂▃▅████████████▇▆▅▃▃▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  7.42 μs         Histogram: frequency by time        11.1 μs <

 Memory estimate: 7.77 KiB, allocs estimate: 2.

In [7]:
MaxPool_v1(input, (2,2), gradient) == MaxPool_v2(input, (2,2), gradient)

true